In [1]:
# To find ECD Control

import sys 
sys.path.append('C:\\Users\\Eesh Gupta\\Documents\\RU Research\\Chakram')


In [4]:
import numpy as np
from qutip import *
from ECD_batch_optimizerV1 import BatchOptimizer
from Simulation_Classes_Single_ModeV4 import *
import matplotlib.pyplot as plt

In [5]:
%load_ext autoreload
%autoreload 2

In [31]:
import pandas as pd
df = None
columns=['time', 'layer', 'rep fid']

In [47]:
#The target oscillator state.
N =15
Fock = 1
psi_i = tensor(basis(2,0), basis(N,0)) # initial state
psi_t = tensor(basis(2,0), basis(N,Fock)) #target state

In [48]:
#Optimization of ECD Circuit parameters (betas, phis, and thetas)
#the optimization options
opt_params = {
'N_blocks' : 1, #circuit depth
'N_multistart' : 10, #Batch size (number of circuit optimizations to run in parallel)
'epochs' : 500, #number of epochs before termination
'epoch_size' : 10, #number of adam steps per epoch
'learning_rate' : 0.01, #adam learning rate
'term_fid' : 0.999, #terminal fidelitiy
'dfid_stop' : 1e-6, #stop if dfid between two epochs is smaller than this number
'beta_scale' : 3.0, #maximum |beta| for random initialization
'initial_states' : [psi_i], #qubit tensor oscillator, start in |g> |0>
'target_states' : [psi_t], #end in |e> |target>.
'name' : 'Fock %d' % Fock, #name for printing and saving
'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
}


#note: optimizer includes pi pulse in every ECD step. However, final ECD step is implemented 
#in experiment as a displacement since the qubit and oscillator should be disentangled at this point.
#So, we ask the optimizer to end in |e> |target> instead of |g>|target>.

#### Filename for angles to be stored

In [49]:
df = None 
fname = 'pulse_times_g01_g10_Oct21'

In [50]:
def main(layers, fname = fname, df = df):
    '''
    Vary layers, get fidelity
    '''
    for layer in layers: 
        #optimizer
        opt_params['N_blocks'] = layer
        opt = BatchOptimizer(**opt_params)
        opt.optimize()
        BO_fid = opt.best_fidelity()
        angles_filename = 'test_opt_angles.txt'
        opt.save_angles(filename = angles_filename)
        
        #pulses
        pulse_sim = ecd_pulse_single_mode(param_file = angles_filename,
                                 kappa = 1/(2e+6)) 
        pulse_sim.get_pulses()
        
        pulse_time = len(pulse_sim.alpha)
        
        df_new = pd.DataFrame([[ pulse_time, layer, BO_fid]], columns=columns)
        if df is None : 
            df = df_new
        else:
            df = df.append(df_new, ignore_index = True)
        df.to_csv(fname, index=False)

In [54]:
layers = [i for i in range(2, 8, 1)]

In [55]:
layers

[2, 3, 4, 5, 6, 7]

In [56]:
main(layers)

None
Start time: 2022-10-21 09:54:25
Tensor("alphas:0", shape=(1, 10), dtype=complex64)
Tensor("alphas:0", shape=(2, 10), dtype=complex64)
 Epoch: 34 / 500 Max Fid: 0.367880 Avg Fid: 0.367880 Max dFid: 0.000001 Avg dFid: 0.000000 Elapsed time: 0:00:04.246430 Remaing time: 0:00:58.201066
 max dFid: 0.000001
dFid stop: 0.000001


 Optimization stopped.  No dfid is greater than dfid_stop

optimization_type: state transfer
N_multistart: 10
N_blocks: 2
term_fid: 0.995
dfid_stop: 1e-06
no_CD_end: False
learning_rate: 0.01
epoch_size: 10
epochs: 500
beta_scale: 3.0
alpha_scale: 1.0
theta_scale: 3.141592653589793
use_displacements: False
use_phase: False
name: Fock 1
comment: 
user_angles: None
N_cav: 15
filename: Fock 1.h5

Best circuit parameters found:
betas:         [-0.1928 -1.89285j -0.61323+0.06246j]
alphas:        [0.+0.j]
phis (deg):    [ 0.      69.50025]
thetas (deg):  [71.78806 71.78809]
Max Fidelity:  0.367880


termination reason: dfid
optimization timestamp (start time): 2022-10

/home/eag190/miniconda3/envs/simulations/lib/python3.9/site-packages/scipy/signal/_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)


---------------------------
Final Displacement: 0.8082848796100593
Modified conditional displacement called
---------------------------
Final Displacement: 9.664088599323192e-05
None
Start time: 2022-10-21 09:54:48
Tensor("alphas:0", shape=(1, 10), dtype=complex64)
Tensor("alphas:0", shape=(4, 10), dtype=complex64)


/tmp/ipykernel_30326/2741616155.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_new, ignore_index = True)


 Epoch: 65 / 500 Max Fid: 0.981403 Avg Fid: 0.888826 Max dFid: 0.000001 Avg dFid: 0.000000 Elapsed time: 0:00:08.769133 Remaing time: 0:00:58.685735
 max dFid: 0.000001
dFid stop: 0.000001


 Optimization stopped.  No dfid is greater than dfid_stop

optimization_type: state transfer
N_multistart: 10
N_blocks: 4
term_fid: 0.995
dfid_stop: 1e-06
no_CD_end: False
learning_rate: 0.01
epoch_size: 10
epochs: 500
beta_scale: 3.0
alpha_scale: 1.0
theta_scale: 3.141592653589793
use_displacements: False
use_phase: False
name: Fock 1
comment: 
user_angles: None
N_cav: 15
filename: Fock 1.h5

Best circuit parameters found:
betas:         [-1.56612+0.03763j  0.02434+1.0128j  -0.65398+0.01572j -0.     +0.j     ]
alphas:        [0.+0.j]
phis (deg):    [  0.      -89.99999  89.99978   0.00022]
thetas (deg):  [ 90.00008 -89.99989  89.99982  89.99994]
Max Fidelity:  0.981402


termination reason: dfid
optimization timestamp (start time): 2022-10-21 09:54:48
timestamp (end time): 2022-10-21 09:54:57
elap

...

/tmp/ipykernel_30326/2741616155.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_new, ignore_index = True)


 Epoch: 10 / 500 Max Fid: 0.998352 Avg Fid: 0.928494 Max dFid: 0.084496 Avg dFid: 0.030891 Elapsed time: 0:00:02.081944 Remaing time: 0:01:42.015267

 Optimization stopped. Term fidelity reached.

optimization_type: state transfer
N_multistart: 10
N_blocks: 7
term_fid: 0.995
dfid_stop: 1e-06
no_CD_end: False
learning_rate: 0.01
epoch_size: 10
epochs: 500
beta_scale: 3.0
alpha_scale: 1.0
theta_scale: 3.141592653589793
use_displacements: False
use_phase: False
name: Fock 1
comment: 
user_angles: None
N_cav: 15
filename: Fock 1.h5

Best circuit parameters found:
betas:         [ 0.92991+1.11229j  0.62484-0.42947j  0.02075+0.75208j -0.75133-0.07144j
  0.97904-1.62472j  1.41153-1.63676j  0.01175-0.11535j]
alphas:        [0.+0.j]
phis (deg):    [  0.       88.63784 103.48551  66.79702 -46.19493  38.19178  34.0309 ]
thetas (deg):  [  90.9986   123.20777   82.30898 -103.57394  -36.25889    1.13154
  -76.63657]
Max Fidelity:  0.998343


termination reason: term_fid
optimization timestamp (start

/home/eag190/miniconda3/envs/simulations/lib/python3.9/site-packages/scipy/signal/_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)


---------------------------
Final Displacement: 0.7585273185220985
Modified conditional displacement called
---------------------------
Final Displacement: 0.7523709177577791
Modified conditional displacement called
---------------------------
Final Displacement: 0.7547185338970385
Modified conditional displacement called
---------------------------
Final Displacement: 1.8968997240866923
Modified conditional displacement called
---------------------------
Final Displacement: 2.1613383379471034
Modified conditional displacement called
---------------------------
Final Displacement: 0.11594115952505771


/tmp/ipykernel_30326/2741616155.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_new, ignore_index = True)
